# Training and Tuning at Scale

#### Amazon SageMaker provides managed distributed training and tuning capabilities to improve training efficiency, and capabilities to organize and track ML experiments at scale. SageMaker enables techniques such as streaming data into algorithms by using pipe mode for training with data at scale and Managed Spot Training for reduced training costs. 

## ML training at scale with SageMaker distributed libraries

#### Two common scale challenges with ML projects are scaling training data and scaling model size. While increased training data volume, model size, and complexity can potentially result in a more accurate model, there is a limit to the data volume and the model size that you can use with a single compute node, CPU, or GPU. Increased training data volumes and model sizes typically result in more computations, and therefore training jobs take longer to finish, even when using powerful compute instances such as Amazon Elastic Compute Cloud (EC2) p3 and p4 instances.

#### Distributed training is a commonly used technique to speed up training when dealing with scale challenges. Training load can be distributed either across multiple compute instances (nodes), or across multiple CPUs and GPUs (devices) on a single compute instance. There are two strategies for distributed training – data parallelism and model parallelism. Their names are a good indication of what is involved with each strategy. With data parallelism, the training data is split up across multiple nodes (or devices). With model parallelism, the model is split up across the nodes (or devices).

#### Mixed-precision training is a popular technique to handle training at scale and reduce training time. Typically used on compute instances equipped with NVIDIA GPUs, mixed-precision training converts network weights from FP32 representation to FP16, calculates the gradients, converts weights back to FP32, multiplies by the learning rate, and finally updates the optimizer weights.

#### In the data parallelism distribution strategy, the ML algorithm or the neural network-based model is replicated on all devices, and each device processes a batch of data. Results from all devices are then combined. In the model parallelism distribution strategy, the model (which is the neural network) is split up across the devices. Batches of training data are sent to all devices so that the data can be processed by all parts of the model. 

#### Both data and model parallelism distribution strategies come with their own complexities. With data parallelism, each node (or device) is trained on a subset of data (called a mini-batch), and a mini-gradient is calculated. However, within each node, a mini-gradient average, with gradients coming from other nodes, should be calculated and communicated to all other nodes. This step is called all reduce, which is a communication overhead that grows as the training cluster is scaled up.

#### While model parallelism addresses the requirements of a model not fitting in a single device's memory by splitting it across devices, partitioning the model across multiple GPUs may lead to under-utilization. This is because training on GPUs is sequential in nature, where only one GPU is actively processing data while the other GPUs are waiting to be activated. To be effective, model parallelism should be coupled with a pipeline execution schedule to train the model across multiple nodes, and in turn, maximize GPU utilization. Now that you know two different distribution strategies, how do you choose between data and model parallelism?

## Choosing between data and model parallelism

##### When choosing a distributed strategy to implement, keep in mind the following:

- Training on multiple nodes inherently causes inter-node communication overhead.
- Additionally, to meet security and regulatory requirements, you may choose to protect the data transmitted between the nodes by enabling inter-container encryption.
- Enabling inter-container encryption will further increase the training time.


#### Due to these reasons, use data parallelism if the trained model can fit in the memory of a single device or node. In situations where the model does not fit in the memory due to its size or complexity, you should experiment further with data parallelism before deciding on model parallelism.

- Tuning the model's hyperparameters: Tuning parameters such as the number of layers of a neural network, or the optimizer to use, affects the model's size considerably.
- Reducing the batch size: Experiment by incrementally reducing the batch size until the model fits in the memory. This experiment should balance out the model's memory needs with optimal batch size. Make sure you do not end up with a suboptimal small batch size just because training with a large batch size takes up most of the device memory.
- Reducing the model input size: If the model input is tabular, consider embedding vectors of reduced dimensions. Similarly, for natural language processing (NLP) models, reduce the input NLP sequence length, and if the input is an image, reduce image resolution.
- Using mixed-point precision: Experiment with mixed-precision training, which uses FP16 representation of weights during gradient calculation, to reduce memory consumption.

## Scaling the compute resources

#### Both the distributed training strategies depend on a cluster of compute resources to spread the training load. When scaling the distributed cluster to meet the training demands, the recommended best practices are as follows:

- First, scale vertically. That is, scale from a single GPU to multiple GPUs on a single instance. For example, let's say you started with the instance type p3.2xlarge, which has a single GPU for training your model, and you find yourself needing a greater number of GPUs to increase the training time. Change the instance type to p3.16xlarge, which has eight GPUs. This will result in a nearly eight-times decrease in the training, a near-linear speedup. Keeping the training job on a single scaled-up instance results in better performance than using multiple instances while keeping the cost low. 

- Next, scale from a single instance to multiple instances. When you reach limits of the instance types offered and still need to scale your training even further, then use multiple instances of the same type, that is, scale from a single p3.16xlarge to two p3.16xlarge instances. This will give you double the compute capacity, going from 8 GPUs on a single instance, to 16 GPUs across two instances. Keep in mind that when you use multiple instances in the training cluster, all instances should be in the same Availability Zone. For example, instances in us-west-2 must all be in us-west-2a or all in us-west-2b. Your training data should also be in the same region, us-west-2.

#### When moving from a single instance to multiple instances, it is recommended that you observe the model convergence and increase the batch size as necessary. Since the batch size you use is split across GPUs, each GPU is processing a lower batch size, which could lead to a high error rate and disrupt the model convergence.

## Automated model tuning with SageMaker hyperparameter tuning

#### Hyperparameter tuning (HPT) helps you find the right parameters to use with your ML algorithm or the neural network to find an optimal version of the model. Amazon SageMaker supports managed hyperparameter tuning, also called automatic model tuning. In this section, we discuss the best practices to consider while configuring hyperparameter jobs on Amazon SageMaker.

#### To execute a SageMaker hyperparameter tuning job, you specify a set of hyperparameters, a range of values to explore for each hyperparameter, and an objective metric to measure the model's performance. Automatic tuning executes multiple training jobs on your training dataset with the ML algorithm and the hyperparameter values to find the best-performing model as measured by the objective metric.

## Step 1 Set up

In [ ]:
import boto3
import pandas as pd

import sagemaker
from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput

from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

## Define Variables

In [ ]:
# set an output path where the trained model will be saved
s3_bucket = 'datascience-environment-notebookinstance--06dc7a0224df'
s3_prefix = 'prepared'
m_prefix = 'hpo'
output_path = 's3://{}/{}/{}/output'.format(s3_bucket, m_prefix, 'xgboost')
content_type = 'text/csv'

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name

# Step 2: Automatic model tuning of the XGBoost algorithm

In [ ]:
#Objective metric to optimize
objective_metric_name = "validation:rmse"
#Objective type.  Here we want to minimize the objective metric
objective_type = 'Minimize'
#Define the hyperparameter ranges to explore
hyperparameter_ranges = {
    "eta": ContinuousParameter(0, 1),
    "min_child_weight": ContinuousParameter(1, 10),
    "alpha": ContinuousParameter(0, 2),
    "max_depth": IntegerParameter(1, 10),
}
#Total training jobs to be run as part of HPT
max_jobs=10
#Maximum number of training jobs to run in parallel
max_parallel_jobs=2

# Step 2.2: Define the estimator and HPT job

In [ ]:
# This line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", region, "1.2-1")

# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"reg:squarederror",
        "num_round":"5"}

# define the data type and paths to the training and validation datasets
train_input = TrainingInput("s3://{}/{}/{}/".format(s3_bucket, s3_prefix, 'train'), content_type=content_type, distribution='ShardedByS3Key')
validation_input = TrainingInput("s3://{}/{}/{}/".format(s3_bucket, s3_prefix, 'validation'), content_type=content_type, distribution='ShardedByS3Key')

#Define the estimator
estimator_hpo = sagemaker.estimator.Estimator(image_uri=xgboost_container, 
                                          hyperparameters=hyperparameters, ##Is this needed?
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.12xlarge', 
                                          volume_size=200, # 5 GB 
                                          output_path=output_path)

#Define the Tuner object
tuner = HyperparameterTuner(
    estimator_hpo, 
    objective_metric_name, 
    hyperparameter_ranges, 
    max_jobs=10,  #Total training jobs to be run as part of HPT
    max_parallel_jobs=2, #Maximum number of training jobs to run in parallel
    objective_type = objective_type
)



## Step 2.3 Run the tuner job

In [ ]:
#Fit method
tuner.fit({'train': train_input, 'validation': validation_input})

tuning_job_name = tuner.latest_tuning_job.name
tuning_job_name

## 2.4 Examine the HPT results

In [ ]:
##Get results from HPT
tuner_results = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)
##Get the results as a dataframe
full_df = tuner_results.dataframe()
##Display results sorted by objective metric values
if len(full_df) > 0:
    df = full_df[full_df["FinalObjectiveValue"] > -float("inf")]
    if len(df) > 0:
        df = df.sort_values("FinalObjectiveValue", ascending=True)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest": min(df["FinalObjectiveValue"]), "highest": max(df["FinalObjectiveValue"])})
        pd.set_option("display.max_colwidth", -1)  # Don't truncate TrainingJobName
    else:
        print("No training jobs have reported valid results yet.")

df

## Step 3 Compare Bayesian (default) and Random tuning strtegies

#### Now lets try HPO with random strategy. In the default tuning method above, uses Bayesian Optimization which is a sequential algorithm that learns from past trainings as the tuning job progresses. This highly limits the level of parallelism.

#### However, with random search you have an option to run all jobs parallel. But a disadvantage is that the random search typically requires running considerably more training jobs to reach a comparable model quality.

#### Please take into consideration your AWS account limit for the number of instances you can launch in parallel.

In [ ]:
#Define the Tuner object
random_tuner = HyperparameterTuner(
    estimator_hpo, 
    objective_metric_name, 
    hyperparameter_ranges, 
    max_jobs=20,  #Total training jobs to be run as part of HPT
    max_parallel_jobs=10, #Maximum number of training jobs to run in parallel
    strategy="Random",
    objective_type = objective_type
)

#Fit method
random_tuner.fit({'train': train_input, 'validation': validation_input})

tuning_job_name = random_tuner.latest_tuning_job.name
tuning_job_name

##Get results from HPT
tuner_results = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)
##Get the results as a dataframe
full_df = tuner_results.dataframe()
##Display results sorted by objective metric values
if len(full_df) > 0:
    df = full_df[full_df["FinalObjectiveValue"] > -float("inf")]
    if len(df) > 0:
        df = df.sort_values("FinalObjectiveValue", ascending=True)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest": min(df["FinalObjectiveValue"]), "highest": max(df["FinalObjectiveValue"])})
        pd.set_option("display.max_colwidth", -1)  # Don't truncate TrainingJobName
    else:
        print("No training jobs have reported valid results yet.")

df

#### Selecting a small number of hyperparameters: HPT is a computationally intensive task, the computational complexity being proportional to the number of hyperparameters you want to tune. SageMaker allows you to specify up to 20 hyperparameters to optimize for a tuning job but limiting your search to a smaller number is likely to give you better results.

#### Selecting a small range for hyperparameters: Along the same lines, the range of values for hyperparameters can significantly affect the success of hyperparameter optimization. Intuitively, you may want to specify a very large range to explore all possible values for a hyperparameter, but you will in fact get better results by limiting your search to a small range of values.

#### Specifying hyperparameter type: For the hyperparameters you want to explore, select the right type from the three types supported—categorical, integer, and continuous. Use the categorical type to test different categorical values for a hyperparameter, such as different optimizers for a neural network. Additionally, you can also use the categorical type when you want to test specific values.

#### For example, for the train_batch_size hyperparameter, instead of exploring a range in a linear fashion, you might want only to evaluate the two values–128 and 256. In this case, you treat the parameter as a categorical value. In contrast, if you want to explore the values for the train_batch_size hyperparameter in a range from a minimum threshold value of 128 to a maximum threshold value of 256, you will use the Integer type. The Integer type allows for greater exploration of the range.

#### If you search a range that spans several orders of magnitude, you can optimize the search by choosing a logarithmic scale for Integer hyperparameters. Finally, choose a continuous parameter if the range of all values to explore, from the lowest to the highest, is relatively small. For example, exploring the learning_rate hyperparameter in the range of 0.0001 and 0.0005 at a linear scale.

#### Enabling warm start: SageMaker HPT supports warm start, which reuses results from one or more prior tuning jobs as a starting point. Configure your HPT job to use warm start to limit the combinations of hyperparameters to search over in the new tuning job. This results in a faster tuning job. Warm start is particularly useful when you want to change the HPT ranges from the previous job or add new hyperparameters.

#### Enabling early stop to save tuning time and costs: With early stop enabled, the individual training jobs launched by the HPT job will terminate early when the objective metric is not improving significantly. After each epoch of training, a running average of the objective metric for all the previous training jobs up to the same epoch is determined and the median of running averages is calculated. If the value of the objective metric for the current training job is worse than the median value, SageMaker stops the current training job.

#### Selecting a small number of concurrent training jobs: SageMaker allows you to execute multiple training jobs concurrently as part of the overall tuning job using the MaxParallelTrainingJobs parameter. On one hand, running more HPT jobs concurrently completes the tuning job quickly. On the other, a tuning job can only find better combinations of hyperparameters through successive rounds of experiments. In the long run, executing a single training job at a time gives the best results with minimum computation time.

#### This is the case when the default Bayesian optimization tuning strategy is used by SageMaker HPO. However, if you have experience with your algorithm and dataset, you can also use the random search strategy natively supported by SageMaker, since it enables concurrency but doesn't require serial rounds of experiments.